In [1]:
import pytorch_lightning as pl
import pytorch_lightning.callbacks as pl_callbacks
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import eq


from eq.data import Catalog, InMemoryDataset, Sequence, default_catalogs_dir

In [30]:
catalog1 = eq.catalogs.SCEDC()
catalog2 = eq.catalogs.ANSS_MultiCatalog(    
    num_sequences=100,
    t_end_days=1*365,
    mag_completeness=4.5,
    minimum_mainshock_mag=6.0,
)

Loading existing catalog from /home/gcl/RA/jonahk/recast/data/SCEDC.
Loading existing catalog from /home/gcl/RA/jonahk/recast/data/ANSS_MultiCatalog.


/home/gcl/RA/jonahk/recast/eq/data/sequence.py:207: UserWarning: Found 1 zero inter-event times in the sequence. This violates fundamental assumptions of TPP models and may lead to incorrect log-likelihood values.
  warnings.warn(


In [31]:

def combine_catalogs_sequences(seqlist):
    sequences = []
    for seq in seqlist:
        sequences.append(seq)
    return InMemoryDataset(sequences=sequences)

def build_seqlist(catalogs):
    train_sequences = []
    for catalog in catalogs:
        for seq in range(len(catalog.train)):
            train_sequences.append(catalog.train[seq])
    return train_sequences

def subtract_magnitudes(sequences, mag_completeness):
    for seq in sequences:
        seq.mag -= mag_completeness


In [32]:
catalogs = []
catalogs.append(catalog1)
catalogs.append(catalog2)


#subtract mag_complete of ANSS from ANSS sequences
#subtract mag_complete of SCEDC from SCEDC sequence
seqlist = build_seqlist(catalogs)
combined_train = combine_catalogs_sequences(seqlist)
# subtract_magnitudes(combined_train.sequences, 4.5)
# subtract_magnitudes(catalog1.val.sequences, 4.5)

In [33]:
combined_train.sequences[0].mag 

tensor([2.2600, 2.3700, 2.1600,  ..., 2.3500, 2.0600, 2.1000])

In [34]:
train = combined_train.get_dataloader( batch_size=5, shuffle=True)
val = catalog1.val.get_dataloader( batch_size=5, shuffle=True)



In [35]:
T = combined_train.sequences[0].t_end
N = np.mean([len(seq) for seq in combined_train])
mag_mean = np.mean([combined_train.sequences[0].mag.mean().item() for seq in combined_train])
tau_mean = T/N
mag_completness = 0.6

anss_double_model = eq.models.RecurrentTPP(
    mag_mean = mag_mean,
    tau_mean = tau_mean,
    mag_completeness = mag_completness,
    learning_rate=1e-3,
)

In [36]:

    # ModelCheckpoints saves the model with the best validation loss
checkpoint = pl_callbacks.ModelCheckpoint(monitor="total_val_loss")

    # EarlyStopping stops training if the validation loss doesn't improve by more than 1e-3 for 20 epochs
early_stopping = pl_callbacks.EarlyStopping(monitor="total_val_loss", patience=10, min_delta=1e-5)

    # RichProgressBar adds a nice and more functional progress bar
progress_bar = pl_callbacks.RichProgressBar()

    # Trainer set up training and validation loops with previous specs
trainer = pl.Trainer(devices=1,max_epochs=100, callbacks=[checkpoint, early_stopping, progress_bar],log_every_n_steps=1)

trainer.fit(anss_double_model, train, val)

trainer.test(anss_double_model, val)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


┏━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name          ┃ Type    ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩
│ 0 │ hypernet_time │ Linear  │  3.2 K │
│ 1 │ hypernet_mag  │ Linear  │     33 │
│ 2 │ rnn           │ GRU     │  3.5 K │
│ 3 │ dropout       │ Dropout │      0 │
└───┴───────────────┴─────────┴────────┘

Trainable params: 6.7 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 6.7 K                                                                                                
Total estimated model params size (MB): 0

Output()

`Trainer.fit` stopped: `max_epochs=100` reached.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    -12.83415699005127     │
│      total_test_loss      │    -12.83415699005127     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': -12.83415699005127, 'total_test_loss': -12.83415699005127}]

In [15]:
#Compare results to single catalog training
single_train = catalog1.train.get_dataloader(batch_size=5, shuffle=True)

T = catalog1.train[0].t_end

N = np.mean([len(seq) for seq in catalog1.train])
mag_mean = np.mean([catalog1.train[0].mag.mean().item() for seq in catalog1.train])
tau_mean = T/N

single_model = eq.models.RecurrentTPP(
    mag_mean = mag_mean,
    tau_mean = tau_mean,
    mag_completeness=catalog1.metadata['mag_completeness'],
    learning_rate=1e-3,
)

    # ModelCheckpoints saves the model with the best validation loss
checkpoint = pl_callbacks.ModelCheckpoint(monitor="total_val_loss")

    # EarlyStopping stops training if the validation loss doesn't improve by more than 1e-3 for 20 epochs
early_stopping = pl_callbacks.EarlyStopping(monitor="total_val_loss", patience=10, min_delta=1e-5)

    # RichProgressBar adds a nice and more functional progress bar
progress_bar = pl_callbacks.RichProgressBar()

    # Trainer set up training and validation loops with previous specs
trainer = pl.Trainer(devices=1,max_epochs=1000, callbacks=[checkpoint, early_stopping, progress_bar],log_every_n_steps=1)

trainer.fit(single_model, single_train, val)

trainer.test(single_model, val)





GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


┏━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name          ┃ Type    ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩
│ 0 │ hypernet_time │ Linear  │  3.2 K │
│ 1 │ hypernet_mag  │ Linear  │     33 │
│ 2 │ rnn           │ GRU     │  3.5 K │
│ 3 │ dropout       │ Dropout │      0 │
└───┴───────────────┴─────────┴────────┘

Trainable params: 6.7 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 6.7 K                                                                                                
Total estimated model params size (MB): 0

Output()

`Trainer.fit` stopped: `max_epochs=1000` reached.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    -13.12653636932373     │
│      total_test_loss      │    -13.12653636932373     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': -13.12653636932373, 'total_test_loss': -13.12653636932373}]